# INSTRUCTIONS

Workflow: Run each cell in order. Put or extract data into relevant folders as defined in section 2. Make remaining folders in drive.  **Cells denoted with * may require extra action.**


```
Folder Structure Suggested for Section 2:
drive/
    train_data/ <- input midis go here
    train_output/
        graphs/ <- train/val accuracy plots go here
        intermed/ <- intermediate weights, preprocessing go here
        stats/ (currently not being used)
        midi/ <- output midis go here
```




# 1 Imports

In [1]:
# Import Data Manip, Debug
import glob
import pickle
import numpy as np
import pandas as pd
import pdb
import matplotlib.pyplot as plt

# Import Music21
!pip install music21
from music21 import converter, instrument, note, chord, stream

# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import CuDNNGRU
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import Flatten
from keras.regularizers import L1L2
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
# stuff needed for colaboratory to connect with drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

# 2 Mount and Set Directories*

In [0]:
# mount drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
DATA_FOLDER = 'drive/Project/train_data/rock_melody'
GRAPHS_FOLDER = 'drive/Project/train_output/graphs'
MIDI_OUTPUT_FOLDER = 'drive/Project/train_output/midi'
INTERMED_FOLDER = 'drive/Project/train_output/intermed'
STATS_FOLDER = 'drive/Project/train_output/stats'

# data specific
ROCK_CORPUS = '/rock_'

# SET THIS TO ONE OF ABOVE (ACTION)
CORPUS = ROCK_CORPUS

# Notewise Root and Rest Learning

# 3 Set Training Parameters*

In [0]:
# SET PARAMETERS (ACTION)
RESTS = False
ROOT_EXTRACTION = False
DURATION_BATCH_SIZE = 256
NOTE_BATCH_SIZE = 128
# SPECIFY PARAMETERS TO TEST AS LIST
DROPOUTS = 0
MODEL_SIZES = 256
EPOCHS = 10

# 4 Preprocess Data (MIDI Data Into Notes Corpus and Duration Corpus)

In [6]:
# Test

notes_corpus = []
durations_corpus = []

for file in glob.glob(DATA_FOLDER + "/*.mid"):
    try:
      print("Extracting MIDI File: ", file)
      midi_stream = converter.parse(file)

      notes = None

      partition = instrument.partitionByInstrument(midi_stream)

      if not RESTS:
        # No rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.flat.notes
      else:
        # With rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.notesAndRests
          
      in_song_notes = []
      in_song_durations = []
      for element in notes:
          in_song_durations.append(element.duration.quarterLength)
          if isinstance(element, note.Note):
              in_song_notes.append(str(element.pitch))
          elif RESTS and isinstance(element, note.Rest):
              in_song_notes.append("R")
          elif isinstance(element, chord.Chord):
              if ROOT_EXTRACTION:
                  in_song_notes.append(element.root().nameWithOctave)
              else:
                  in_song.append('.'.join(str(n) for n in element.normalOrder))
    except:
      pass
    
    notes_corpus.append(in_song_notes)
    durations_corpus.append(in_song_durations)
            
# Write
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'wb+') as filepath:
    pickle.dump(notes_corpus, filepath)
    
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'wb+') as filepath:
    pickle.dump(durations_corpus, filepath)

Extracting MIDI File:  drive/Project/train_data/rock_melody/2955.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2949.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2950.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2953.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2952.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2951.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2948.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2947.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2946.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2954.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2945.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2939.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2936.mid
Extracting MIDI File:  drive/Project/train_data/rock_melody/2937.mid
Extracting MIDI File:  drive/Proje

# 5 Preprocess Corpus Into Train Data

## 5.1 Notes Corpus

In [7]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'rb') as filepath:
    notes_corpus = pickle.load(filepath)
    
# If doing learning for one song at a time only
flattened_notes_corpus = []
for song_notes in notes_corpus:
    flattened_notes_corpus += song_notes

vocab_size = len(set(flattened_notes_corpus))
print(vocab_size)

# Produce input output sequences
window_size = 60
note_sequence_input = []
next_note_output = []

notes = sorted(set(flattened_notes_corpus))
note2int = dict((note, num) for num, note in enumerate(notes))

for i in range(0, len(notes_corpus)):
    for j in range(0, len(notes_corpus[i]) - window_size):
        current_sequence = [note2int[note] for note in notes_corpus[i][j:window_size+j]]
        next_note = note2int[notes_corpus[i][window_size+j]]
        note_sequence_input.append(current_sequence)
        next_note_output.append(next_note)

87


In [8]:
# check train and label shapes
training_data = np.reshape(note_sequence_input, (len(note_sequence_input), window_size , 1))
training_data = training_data / float(vocab_size)
print('Train shape: ' + str(training_data.shape))
training_label = np_utils.to_categorical(next_note_output)
print('Label shape: ' + str(training_label.shape))

Train shape: (27013, 60, 1)
Label shape: (27013, 86)


## 5.2 Durations Corpus

In [9]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'rb') as filepath:
    duration_corpus = pickle.load(filepath)

# Learn on one song at a time
flattened_duration_corpus = []
for song_durations in duration_corpus:
    flattened_duration_corpus += song_durations
    
# import collections
# counter = collections.Counter(flattened_duration_corpus)
# print(counter)


duration_vocab_size = len(set(flattened_duration_corpus))
print(duration_vocab_size)

# Produce input output sequences
duration_window_size = 8
duration_sequence_input = []
next_duration_output = []

durations = sorted(set(flattened_duration_corpus))
duration2int = dict((duration, num) for num, duration in enumerate(durations))

print(duration2int)
# # Write
# with open(INTERMED_FOLDER + "/edm_duration_counter", 'wb+') as filepath:
#     pickle.dump(counter, filepath)

for i in range(0, len(duration_corpus)):
    for j in range(0, len(duration_corpus[i]) - duration_window_size):
        current_duration_sequence = [duration2int[note] for note in duration_corpus[i][j:duration_window_size+j]]
        next_duration = duration2int[duration_corpus[i][duration_window_size+j]]
        duration_sequence_input.append(current_duration_sequence)
        next_duration_output.append(next_duration)

1218
{0.0: 0, Fraction(1, 12): 1, Fraction(1, 6): 2, 0.25: 3, Fraction(1, 3): 4, Fraction(5, 12): 5, 0.5: 6, Fraction(7, 12): 7, Fraction(2, 3): 8, 0.75: 9, Fraction(5, 6): 10, Fraction(11, 12): 11, 1.0: 12, Fraction(13, 12): 13, Fraction(7, 6): 14, 1.25: 15, Fraction(4, 3): 16, Fraction(17, 12): 17, 1.5: 18, Fraction(19, 12): 19, Fraction(5, 3): 20, 1.75: 21, Fraction(11, 6): 22, Fraction(23, 12): 23, 2.0: 24, Fraction(25, 12): 25, 2.25: 26, Fraction(7, 3): 27, Fraction(29, 12): 28, 2.5: 29, Fraction(31, 12): 30, Fraction(8, 3): 31, 2.75: 32, Fraction(17, 6): 33, Fraction(35, 12): 34, 3.0: 35, Fraction(37, 12): 36, 3.25: 37, Fraction(10, 3): 38, 3.5: 39, Fraction(43, 12): 40, Fraction(11, 3): 41, 3.75: 42, 4.0: 43, Fraction(25, 6): 44, 4.25: 45, Fraction(13, 3): 46, Fraction(53, 12): 47, 4.5: 48, Fraction(14, 3): 49, 4.75: 50, Fraction(29, 6): 51, 5.0: 52, Fraction(61, 12): 53, Fraction(31, 6): 54, 5.25: 55, 5.5: 56, Fraction(17, 3): 57, 5.75: 58, 6.0: 59, 6.25: 60, Fraction(19, 3): 6

In [10]:
# check train and label shapes
duration_training_data = np.reshape(duration_sequence_input, (len(duration_sequence_input), duration_window_size , 1))
duration_training_data = duration_training_data / float(duration_vocab_size)
print('Train shape: ' + str(duration_training_data.shape))
duration_training_label = np_utils.to_categorical(next_duration_output)
print('Label shape: ' + str(duration_training_label.shape))

Train shape: (45538, 8, 1)
Label shape: (45538, 1216)


# 6 Train

## 6.1 Helpers to Create Model

In [0]:
def create_model(network_input, n_vocab, model_size, dropout):
  model = Sequential()
  reg = L1L2(0, 0)
  model.add(LSTM(
      model_size,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True,
      dropout=dropout, recurrent_dropout=0.3
  ))
  model.add(CuDNNLSTM(model_size, return_sequences=True, kernel_regularizer=reg))
  model.add(Dropout(dropout))
  model.add(CuDNNLSTM(model_size, kernel_regularizer=reg))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_callback_list(model_size, dropout, model_type):
  filepath = INTERMED_FOLDER + '/%skarpathy-model-weights-%s-%s-%s.hdf5' % (CORPUS, model_type, model_size, dropout)
  model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
  )
  return [model_checkpoint], filepath

# acc history
def setup_plot(dropout, size):
  plt.title('Model Accuracy vs. Epoc with Dropout=%s Size=%s' % (dropout, size))
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  
def plot_history(history, model_type, dropout, size):
  plt.plot(history.history['acc'], label="%s train accuracy" % model_type)
  plt.plot(history.history['val_acc'], label="%s val accuracy" % model_type)

def save_plot(file_path):
  plt.legend()
  plt.savefig(file_path)
  plt.clf()
  
def predict_duration(model, WEIGHT_PATH):
  # Prediction
  model.load_weights(WEIGHT_PATH)
  starting_sequence = np.random.randint(219, size=duration_window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  duration2note = dict((num, note) for num, note in enumerate(durations))
  print (duration2note)
  
  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(duration_vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Duration: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Duration: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Duration: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

      prediction_values = np.arange(len(prediction[0]))
      prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))

      # Most probable note prediction
#       index = np.argmax(prediction)
#       note_instance = duration2note[index]
#       prediction_output.append(note_instance)
      index = np.random.choice(prediction_values, 1, p=prediction_prob)
      note_instance = duration2note[int(index[0])]
      prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]

  # Write
  with open(INTERMED_FOLDER + ("%sduration_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  
  return prediction_output

def predict_note(model, WEIGHT_PATH):
  model.load_weights(WEIGHT_PATH)
  # Prediction
  starting_sequence = np.random.randint(219, size=window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  int2note = dict((num, note) for num, note in enumerate(notes))
  print (int2note)

  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Note: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Note: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Note: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

#       prediction_values = np.arange(len(prediction[0]))
#       prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))
#       print(prediction_prob)

      # Most probable note prediction
      index = np.argmax(prediction)
      note_instance = int2note[index]
      prediction_output.append(note_instance)

      # Predict based on prob dist
#       index = np.random.choice(prediction_values, 1, p=prediction_prob)
  #     print(index[0])
  #     print(type(index[0]))
#       note_instance = int2note[int(index[0])]
#       prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]
  # Write
  with open(INTERMED_FOLDER + ("%snotes_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  return prediction_output

def output_midi(prediction_output, duration_prediction_output, dropout, model_size, option):
  offset = 0
  output_notes = []
  for pattern, duration in zip(prediction_output, duration_prediction_output):

      if ('.' in pattern) or pattern.isdigit():
          chord_array = pattern.split('.')
          chord_notes = []
          for note_instance in chord_array:
              note_object = note.Note(int(note_instance))
              note_object.duration.quarterLength = duration
              note_object.storedInstrument = instrument.Piano()
              chord_notes.append(note_object)
          chord_object = chord.Chord(chord_notes)
          chord_object.offset = offset
          output_notes.append(chord_object)
      elif 'R' == pattern:
          note_object = note.Rest()
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          output_notes.append(note_object)
      else:
          note_object = note.Note(pattern)
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          note_object.storedInstrument = instrument.Piano()
          output_notes.append(note_object)

      offset += 0.5

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp=MIDI_OUTPUT_FOLDER + CORPUS + '%s_%s_%s.mid' % (option, dropout, model_size))
  print('\nWrote midi...')

## 6.2 Preprocessing Optimization

In [13]:
notes_histories = {}
duration_histories = {}

dropout, model_size = DROPOUTS, MODEL_SIZES
setup_plot(dropout, model_size)
print('Running duration training on notewise with rests:%s and root extraction:%s' % (RESTS, ROOT_EXTRACTION))
duration_callbacks, duration_weight_path = create_callback_list('duration', dropout, model_size)
duration_model = create_model(duration_training_data, duration_training_label.shape[1], model_size, dropout)
print(duration_model.summary())
duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=DURATION_BATCH_SIZE, callbacks=duration_callbacks, validation_split=0.2)
plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
# output intermed duration
duration_prediction = predict_duration(duration_model, duration_weight_path)

print('\n\nRunning notes training on d:%s s:%s' % (dropout, model_size))
notes_callbacks, note_weight_path = create_callback_list('notes', dropout, model_size)
notes_model = create_model(training_data, training_label.shape[1], model_size, dropout)
notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=NOTE_BATCH_SIZE, callbacks=notes_callbacks, validation_split=0.2)
plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
save_plot(GRAPHS_FOLDER + CORPUS + 'dropout=%s_size=%s' % (dropout, model_size))
# output intermed notes
note_prediction = predict_note(notes_model, note_weight_path)

# output final midi
output_midi(note_prediction, duration_prediction, dropout, model_size, "notewise-chord")

Running duration training on notewise with rests:False and root extraction:False
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 8, 256)            264192    
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 8, 256)            526336    
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 256)            0         
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_____________________________________________________________

<Figure size 576x396 with 0 Axes>